# Evaluation of Formulas from First Order Logic

In this notebook we show how formulas from *first order logic* can be evaluated in Python.

## The Axioms of Group Theory

To have a nontrivial example of formulas, we use the formulas from 
[group theory](https://en.wikipedia.org/wiki/Group_theory).  
A [group](https://en.wikipedia.org/wiki/Group_(mathematics)) is defined as a triple 
$$ \langle G, \mathrm{e}, * \rangle $$
where 
- $G$ is a non-empty set,
- $\mathrm{e}$ is an element from $G$, and
- $*:G \times G \rightarrow G$ is a binary function on $G$.
- Furthermore, the following axioms have to be satisfied:
  * $\forall x: \mathrm{e} * x = x$,
  * $\forall x: \exists{y}: y * x = \mathrm{e}$,
  * $\forall x: \forall y: \forall z: (x * y) * z = x * (y * z)$.
- A group is <em style="color:blue">commutative</em> if, additionally, the following formula is satisfied:
  $$\forall x: \forall y: x * y = y * x. $$

The notebook `FOL-Parser.ipynb` contains a notebook implementing a parser for first order logic.

In [1]:
%%capture
%run FOL-Parser.ipynb

We import a parser for FOL formulas.  This parser distinguishes between variables and function symbol as follows:
- A word starting with a lower case letter is interpreted as a *variable*.
- A word starting with an upper case letter is assumed to be a *function* or 
  *predicate symbol*.

Therefore, we represent the symbols from group theory as follows:
- The neutral element $\mathrm{e}$ of group theory is represented as the nullary function symbol `E`.
- As our parser does not support using the symbol $*$ as a binary operator, we will use the function symbol     
  `Multiply` to represent this operator.
- The predicate symbol $=$ is repesented as `Equals`

Then the formulas of group theory can be represented as follows:

In [2]:
G1 = '∀x:Equals(Multiply(E(),x),x)'

In [3]:
G2 = '∀x:∃y:Equals(Multiply(y,x),E())'

In [4]:
G3 = '∀x:∀y:∀z:Equals(Multiply(Multiply(x,y),z), Multiply(x,Multiply(y,z)))'

In [5]:
G4 = '∀x:∀y:Equals(Multiply(x,y), Multiply(y,x))'

The function $\texttt{parse}(s)$ takes a string $s$ and converts it into a nested tuple.

In [6]:
def parse(s):
    "Parse string s as fol formula."
    p = LogicParser(s)
    return p.parse()

In [7]:
F1 = parse(G1)
F1

('∀', 'x', ('Equals', ('Multiply', ('E',), 'x'), 'x'))

In [8]:
F2 = parse(G2)
F2

('∀', 'x', ('∃', 'y', ('Equals', ('Multiply', 'y', 'x'), ('E',))))

In [9]:
F3 = parse(G3)
F3

('∀',
 'x',
 ('∀',
  'y',
  ('∀',
   'z',
   ('Equals',
    ('Multiply', ('Multiply', 'x', 'y'), 'z'),
    ('Multiply', 'x', ('Multiply', 'y', 'z'))))))

In [10]:
F4 = parse(G4)
F4

('∀',
 'x',
 ('∀', 'y', ('Equals', ('Multiply', 'x', 'y'), ('Multiply', 'y', 'x'))))

## A Structure for Group Theory

The smallest non-trivial group has just two elements.  Therefore, we can define the universe `U` as follows:

In [11]:
U = { 0, 1 }

Next, we need to define the nullary function that represents the nullary function `E`.  We define this function as a dictionary mapping the empty tuple into the element `0`. 

In [12]:
NeutralElement = { (): 0 }

The binary function symbol `Multiply` is implemented as the dictionary `Product`:

In [13]:
Product = { (0, 0): 0,  (0, 1): 1,  (1, 0): 1,  (1, 1): 0 }

The predicate symbol `Equals` is implemented as the binary relation `Identity`.

In [14]:
Identity = { (x, x) for x in U }
Identity

{(0, 0), (1, 1)}

Now the interpretation $\mathcal{J}$ can be implemented as a dictionary mapping symbols to dictionaries that interpret these symbols. 

In [15]:
J = { "E": NeutralElement, "Multiply": Product, "Equals": Identity }

Next, we define the *first order structure* $\mathcal{S}$ as the pair $(\mathcal{U}, \mathcal{J})$.

In [16]:
S = (U, J)
S

({0, 1},
 {'E': {(): 0},
  'Multiply': {(0, 0): 0, (0, 1): 1, (1, 0): 1, (1, 1): 0},
  'Equals': {(0, 0), (1, 1)}})

Finally, we define an *interpretation* $\mathcal{I}$ of the variables $x$, $y$, and $z$. 

In [17]:
I = { "x": 0, "y": 1, "z": 0 } 
I

{'x': 0, 'y': 1, 'z': 0}

## Functions to Evaluate Formulas

In Python, if we precede a variable name with the asteriks `*` in an assignment, then it can consume an arbitrary number of elements.
In the code below, the variable `R` collects all elements from the list `L` with the exception of the first element, which is assigned to `x`. 

In [18]:
L = [1, 2, 3, 4]
x, *R = L
x, R

(1, [2, 3, 4])

The procedure $\texttt{evalTerm}(t, \mathcal{S}, \mathcal{I})$ evaluates the term $t$ in the structure $\mathcal{S}$ using the variable assignment $\mathcal{I}$.

In [19]:
def evalTerm(t, S, I):
    if isinstance(t, str):  # t is a variable
        return I[t]
    _, J     = S      # J is the dictionary of interpretations
    f, *Args = t      # function symbol and list of arguments
    fJ       = J[f]   # interpretation of function symbol
    ArgVals  = tuple(evalTerm(arg, S, I) for arg in Args) # recursively evaluate arguments
    return fJ[ArgVals]

In [20]:
t = parse('Multiply(E(),x)')
t

('Multiply', ('E',), 'x')

In [21]:
evalTerm(t, S, I)

0

This procedure evaluates the atomic formula a in the structure S using the variable assignment I.

In [22]:
def evalAtomic(a, S, I):
    _, J     = S     # J is the dictionary of interpretations
    p, *Args = a     # predicate symbol and arguments
    pJ       = J[p]  # interpretation of predicate symbol
    ArgVals  = tuple(evalTerm(arg, S, I) for arg in Args)
    return ArgVals in pJ

In [23]:
f = parse('Equals(Multiply(E(),x),x)')
f

('Equals', ('Multiply', ('E',), 'x'), 'x')

In [24]:
evalAtomic(f, S, I)

True

Given a variable assignment $\mathcal{I}$, a variable $x$, and an element $c$ from the universe $\mathcal{U}$, the function $\texttt{modify}(\mathcal{I}, x, c)$ computes the variable assignment $\mathcal{I}[x/c]$ which is defined for all variables $y$ as follows:
$$ I[x/c](y) = \left\{ \begin{array}{ll}
                        c     & \mbox{if $x = y$,}  \\
                        I(y)  & \mbox{otherwise.}
                        \end{array}
               \right.
$$

In [25]:
def modify(I, x, c):
    J = I.copy() # do not modify I
    J[x] = c
    return J

Given a first order logic formula $F$, a structure $\mathcal{S}$, and a variable assignment $\mathcal{I}$, the function $\texttt{evalFormula}(F, \mathcal{S}, \mathcal{I})$ computes the truth value of the formula $F$.

In [26]:
def evalFormula(F, S, I):
    U, _ = S # U is the universe
    match F:
        case ('⊤', ):     return True
        case ('⊥', ):     return False
        case ('¬', G):    return not evalFormula(G, S, I)
        case ('∧', G, H): return evalFormula(G, S, I) and evalFormula(H, S, I)
        case ('∨', G, H): return evalFormula(G, S, I) or evalFormula(H, S, I)
        case ('→', G, H): return not evalFormula(G, S, I) or evalFormula(H, S, I)
        case ('↔', G, H): return evalFormula(G, S, I) == evalFormula(H, S, I)
        case ('∀', x, G): return all(evalFormula(G, S, modify(I, x, c)) for c in U)
        case ('∃', x, G): return any(evalFormula(G, S, modify(I, x, c)) for c in U)
    return evalAtomic(F, S, I) 

## Checking whether $\mathcal{S}$ is a Group

In [27]:
print(f"evalFormula({G1}, S, I) = {evalFormula(F1, S, I)}")
print(f"evalFormula({G2}, S, I) = {evalFormula(F2, S, I)}")
print(f"evalFormula({G3}, S, I) = {evalFormula(F3, S, I)}")
print(f"evalFormula({G4}, S, I) = {evalFormula(F4, S, I)}")

evalFormula(∀x:Equals(Multiply(E(),x),x), S, I) = True
evalFormula(∀x:∃y:Equals(Multiply(y,x),E()), S, I) = True
evalFormula(∀x:∀y:∀z:Equals(Multiply(Multiply(x,y),z), Multiply(x,Multiply(y,z))), S, I) = True
evalFormula(∀x:∀y:Equals(Multiply(x,y), Multiply(y,x)), S, I) = True


This shows that the structure $\mathcal{S}$ defined above is indeed a group.  Furthermore, it is a *commutative* group.

## Another Example

Let's show that the formula $\forall x: \exists y:p(x,y) \rightarrow \exists y:\forall x:p(x,y)$ is not *universally valid*, i.e. let's show the following:
$$ \not\models \forall x: \exists y:p(x,y) \rightarrow \exists y:\forall x:p(x,y) $$

In [28]:
G = '∀x:∃y:P(x,y)→∃y:∀x:P(x,y)'

In [29]:
F = parse(G)
F

('→',
 ('∀', 'x', ('∃', 'y', ('P', 'x', 'y'))),
 ('∃', 'y', ('∀', 'x', ('P', 'x', 'y'))))

Our aim is to construct a structure $\mathcal{S} = \langle\mathcal{U}, \mathcal{J} \rangle$  such that 
$$\mathcal{S}(F) = \mathtt{False}.$$ 

In [30]:
U = {0, 1}

In [31]:
pJ = { (0, 0), (1, 1) }

In [32]:
J = { 'P': pJ }

In [33]:
S = (U, J)

In [34]:
I = { 'x': 0, 'y': 0 }

In [35]:
evalFormula(F, S, I)

False